# 리뷰 정제

In [2]:
import pandas as pd

In [3]:
PUBG_review = pd.read_csv("32-0. PUBG_reviews.csv", encoding="utf-8-sig")

In [4]:
import re
PUBG_review["닉네임"] = PUBG_review["닉네임"].str.replace("https://steamcommunity.com/", "", regex=False)
PUBG_review["닉네임"] = PUBG_review["닉네임"].str.extract(r'(?:id/|profiles/)([^/]+)')

In [5]:
PUBG_review["작성일"] = PUBG_review["작성일"].str.replace("Posted: ", "", regex=False)

In [6]:
PUBG_review["본문"] = PUBG_review["본문"].str.replace(r'^Posted:.*?\n', '', regex=True)

In [9]:
def parse_date(x):
    try:
        # day month 형식 (ex: "13 July")
        return pd.to_datetime(x + " 2025", format="%d %B %Y")
    except:
        try:
            # month day 형식 (ex: "June 19")
            return pd.to_datetime(x + " 2025", format="%B %d %Y")
        except:
            return pd.NaT

PUBG_review["작성일"] = PUBG_review["작성일"].apply(parse_date).dt.strftime("%Y-%m-%d")

In [10]:
PUBG_review['추천 여부'] = PUBG_review['추천 여부'].replace({'Not Recommended': 0, 'Recommended': 1})

C:\Users\user\AppData\Local\Temp\ipykernel_9460\3985482955.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  PUBG_review['추천 여부'] = PUBG_review['추천 여부'].replace({'Not Recommended': 0, 'Recommended': 1})


In [11]:
PUBG_review["플레이 시간"] = PUBG_review["플레이 시간"].str.replace(" on record", "", regex=False)

In [12]:
PUBG_review.head()

,닉네임,작성일,본문,추천 여부,플레이 시간
0,76561199331411823,2025-07-04,那么好的一个游戏，全被挂毁了，官方的不整治 ，我身边的朋友都脱坑了，500强的榜单，全是KD...,0,"1,262.1 hrs"
1,76561198992756134,2025-07-12,如果这个游戏没有外挂，我想象不出来得有多好玩,1,310.2 hrs
2,76561199544512016,2025-07-26,"SB,全是挂，举报封三天，蓝洞全体工作人员我操死你的妈",0,"2,138.4 hrs"
3,76561199731852255,2025-07-13,没更新女团前很流畅，更新女团后太鸡巴卡了 掉帧严重！还玩个鸡八,1,368.9 hrs
4,sevenbear0719,2025-08-12,外挂基本杀死了PUBG的绿色玩家，曾几何时大家为了迎合机制拼命的想办法存活可谓玩法极为丰富，...,0,"2,441.5 hrs"


In [13]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    # 줄바꿈, 탭 제거
    text = re.sub(r'\s+', ' ', str(text))
    # 특수문자 제거 (영문/숫자/공백만 남기기)
    text = re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    # 한 글자 단어 제거 (a, I 제외)
    text = ' '.join([w for w in text.split() if len(w) > 1 or w.lower() in ['i', 'a']])
    return text.strip()

PUBG_review["본문"] = PUBG_review["본문"].apply(clean_text)

In [14]:
# 빈 행 (NaN + 공백) 삭제
PUBG_review = PUBG_review[~(PUBG_review["본문"].isna() | PUBG_review["본문"].str.strip().eq(""))]

# 인덱스 리셋
PUBG_review = PUBG_review.reset_index(drop=True)

In [15]:
from googletrans import Translator
import time

# 번역기 초기화
translator = Translator()

# 번역 결과 저장
translated_results = []

for idx, row in PUBG_review.iterrows():
    text = row["본문"]

    if pd.isna(text) or not str(text).strip():
        translated_text = ""
    else:
        try:
            result = translator.translate(text, src='auto', dest='en')
            translated_text = result.text.strip()
        except Exception as e:
            print(f"[에러] {idx}번 번역 실패: {e}")
            translated_text = "[번역 실패]"
    
    print(f"{idx+1}/{len(PUBG_review)} | {row['닉네임']} → {translated_text}")
    translated_results.append(translated_text)

    time.sleep(0.5)  # API 차단 방지 대기

# ds_reviews에 바로 열 추가
PUBG_review["본문_번역"] = translated_results

# 전체 DataFrame 저장
PUBG_review.to_csv("35-1. PUBG_reviews_정제.csv", index=False, encoding='utf-8-sig')
print("번역 열 추가 및 저장 완료 → 35-1. PUBG_reviews_정제.csv")

1/18452 | 76561199331411823 → 500 KD DMA 100 99 DMA
2/18452 | 76561199544512016 → SB
3/18452 | sevenbear0719 → PUBG PUBG 500 1V4 DMA PUBG STEAM 10
4/18452 | 76561198813059052 → 2018 15 steam EVOLVER ID 2018 QBZ 2019 G36C 2020 2020 10 2020 12 PVPVE 2021 S12K 2021 M24 AWM 2022 98 2022 UMP Beryl 2024 M4 M4 Beryl M4 PUBG steam 3000 by EVOLVER SHA
5/18452 | 76561199127926809 → I remember when PUBG was insanely popular on Twitch and everyone around the world was playing like crazy Made some amazing memories with old friends back then They ve updated the game since and it still one of the best survival fight games out there
6/18452 | 76561198064297644 → Because I kill no one I play badly my team denounces me system intended that I should take a permanent bam there man toxic that cursed me with monaca trash this still playing a positive one would really agree with racism in a toxic community I signed the box of received product because in the poca I paid for the game
7/18452 | KingBjornIronsid

In [17]:
PUBG_review = PUBG_review.drop(columns=["본문"])[["닉네임", "작성일", "본문_번역", "추천 여부", "플레이 시간"]]

In [18]:
PUBG_review.to_csv("35-1. PUBG_reviews_정제.csv", index=False, encoding='utf-8-sig')
print("번역 열 추가 및 저장 완료 → 35-1. PUBG_reviews_정제.csv")

번역 열 추가 및 저장 완료 → 35-1. PUBG_reviews_정제.csv
